### Setting Up Environment

In [1]:
!pip install -qU pip
!pip install -qU openai
!pip install -qU google-genai
!pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv
import openai
from google import genai

load_dotenv("../../.env", override=True)

True

In [3]:
#os.environ["OPENAI_API_KEY"] = "<the key>"
#openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

In [7]:
model="gpt-5-nano"

def get_completion(prompt, model="gpt-5-nano"):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
        ]
    response = client.chat.completions.create(model=model,
                                              messages=messages,
                                              )
    return response.choices[0].message.content

# I - Prompting Principles

### Principle 1: Write clear and specific instructions
#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [8]:
text = f"""
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""

response = get_completion(prompt, model=model)
print(response)

Provide clear, specific instructions (detailed prompts) to guide the model and reduce irrelevant outputs, and recognize that longer prompts can offer more clarity and context for better results.


#### Tactic 2: Ask for a structured output
- JSON, HTML, CSV, etc.

In [9]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON format with the following keys:
book_id, title, author, genre.
"""

response = get_completion(prompt)
print(response)

[
  { "book_id": "B101", "title": "The Clockmaker's Mirage", "author": "Iris Calder", "genre": "Fantasy" },
  { "book_id": "B102", "title": "Lanterns Over Saltwater City", "author": "Darian Kuehl", "genre": "Historical Fiction" },
  { "book_id": "B103", "title": "Garden of Static Rivers", "author": "Mara Liao", "genre": "Science Fiction" }
]


In [10]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in CSV format with the following keys:
book_id, title, author, genre.
"""

response = get_completion(prompt)
print(response)

book_id,title,author,genre
B001,The Ember Gate,Astra Nyx,Fantasy
B002,Midnight Echoes in the Library,Calder Thorne,Mystery
B003,The Quantum Orchard,Lira Solenne,Science Fiction


In [11]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide the code to create a Pandas Dataframe with the following columns:
book_id, title, author, genre.
"""

response = get_completion(prompt)
print(response)

Here are three made-up books:

- Whispers Beneath the Glass Ocean — Lila Thorne — Fantasy
- The Clockmaker's Forbidden Recipe — Jasper Kline — Mystery
- Red Lanterns over the Silent Reef — Aria Nygaard — Science Fiction

Python code to create a Pandas DataFrame with the specified columns:

import pandas as pd

books = [
    {"book_id": 101, "title": "Whispers Beneath the Glass Ocean", "author": "Lila Thorne", "genre": "Fantasy"},
    {"book_id": 102, "title": "The Clockmaker's Forbidden Recipe", "author": "Jasper Kline", "genre": "Mystery"},
    {"book_id": 103, "title": "Red Lanterns over the Silent Reef", "author": "Aria Nygaard", "genre": "Science Fiction"},
]

df = pd.DataFrame(books, columns=["book_id", "title", "author", "genre"])
print(df)


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [12]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Boil water.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Remove the tea bag.
Step 6 - Add sugar or milk to taste (optional).
Step 7 - Enjoy your cup of tea.


In [13]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \
walk in the park. The flowers are blooming, and the \
trees are swaying gently in the breeze. People \
are out and about, enjoying the lovely weather. \
Some are having picnics, while others are playing \
games or simply relaxing on the grass. It's a \
perfect day to spend time outdoors and appreciate the \
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""

response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting

In [14]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \
valley flows from a modest spring; the \
grandest symphony originates from a single note; \
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""

response = get_completion(prompt)
print(response)

Resilience is the river that keeps flowing when the banks crumble; it is the note that returns after a missed beat; it is the thread that, stubbornly, stitches a torn cloth into a stronger pattern.


### Principle 2: Give the model time to “think”

#### Tactic 1: Specify the steps required to complete a task

In [15]:
text = f"""
In a charming village, siblings Jack and Jill set out on \
a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune \
struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""

# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Siblings Jack and Jill venture to fetch water from a hilltop well, stumble down the hill but return home safely with their adventurous spirits undimmed.
Les frères et sœurs Jack et Jill s'aventurent pour chercher de l'eau dans un puits perché au sommet d'une colline, trébuchent en descendant la colline, mais rentrent chez eux sains et saufs, leurs esprits aventureux demeurent intacts.
Names in the French summary: Jack, Jill
{"french_summary": "Les frères et sœurs Jack et Jill s'aventurent pour chercher de l'eau dans un puits perché au sommet d'une colline, trébuchent en descendant la colline, mais rentrent chez eux sains et saufs, leurs esprits aventureux demeurent intacts.", "num_names": 2}


#### Ask for output in a specified format

In [16]:
prompt_2 = f"""
Your task is to perform the following actions:
1 - Summarize the following text delimited by
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""

response = get_completion(prompt_2)
print("Completion for prompt 2:")
print(response)

Completion for prompt 2:
Text: <
In a charming village, siblings Jack and Jill set out on a quest to fetch water from a hilltop well. As they climbed, singing joyfully, misfortune struck—Jack tripped on a stone and tumbled down the hill, with Jill following suit. Though slightly battered, the pair returned home to comforting embraces. Despite the mishap, their adventurous spirits remained undimmed, and they continued exploring with delight.
>
Summary: Siblings Jack and Jill embark on a quest to fetch water from a hilltop well, stumble and tumble on the way, but return home unscathed and undeterred, continuing their adventures with joy.
Translation: Jack et Jill, deux frères et sœurs, partent en quête d'eau dans un puits perché au sommet d'une colline; ils trébuchent et dévalent la pente sur le chemin, mais rentrent chez eux indemnes et résolus à poursuivre leurs aventures, continuant leurs explorations avec joie.
Names: ["Jack", "Jill"]
Output JSON: {"french_summary": "Jack et Jill, de

#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [17]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

response = get_completion(prompt)
print(response)

Not correct.

Error: The maintenance cost per square foot is $10, not $100. The maintenance term should be 100,000 + 10x, not 100,000 + 100x.

Correct total cost as a function of x (square feet):
C(x) = land + panels + maintenance
     = 100x + 250x + (100,000 + 10x)
     = 360x + 100,000

So the first-year cost is 360x + 100,000. For example, at x = 1,000 sq ft, C(1000) = 360,000 + 100,000 = 460,000.


#### Note that the student's solution is actually not correct, and the model spotted this.
#### However, sometimes the models can be fooled by the complexity of the prompt. We can fix this by instructing the model to work out its own solution first.

In [18]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""

response = get_completion(prompt)
print(response)

Question:
```
I'm building a solar power installation and I need help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot
What is the total cost for the first year of operations as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
```
Let x be the size of the installation in square feet.
Costs:
- Land: 100x
- Solar panels: 250x
- Maintenance: 100,000 + 10x
Total cost: 100x + 250x + (100,000 + 10x) = (100 + 250 + 10)x + 100,000 = 360x + 100,000
```
Is the student's solution the same as actual solution just calculated:
```
no
```
Student grade:
```
incorrect
```


## Model Limitations: Hallucinations


In [19]:
prompt = f"""
What is the 344th number in a fibonacci sequence? Create a python program, run it and give me the answer.
Make sure to output the answer at the end after running yourself the rproram!
"""

response = get_completion(prompt)
print(response)

I can’t run code in this chat, so I can’t produce the exact 344th Fibonacci number here. I can give you a clean Python program you can run locally to get the exact value. I’m using the standard F0 = 0, F1 = 1 convention, so the 344th Fibonacci number is F(344).

Python program (fast and simple, and it handles big integers natively):
- If you’re using a standard Python 3.x environment:
  - Copy this into a file (e.g., fib344.py) and run python fib344.py

def fib(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a

print(fib(344))

If you prefer a faster approach for even larger n, you can use the fast-doubling method (still simple to implement):

def fib_fast_doubling(n):
    if n == 0:
        return (0, 1)
    else:
        a, b = fib_fast_doubling(n // 2)
        c = a * (2 * b - a)
        d = a * a + b * b
        if n % 2 == 0:
            return (c, d)
        else:
            return (d, c + d)

def fib(n):
    return fib_fast_doubling(n)[0]

print(fi

# II - Iterative Prompt Development
Let's iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

In [20]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture,
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100)
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black,
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities:
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [21]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

Introducing the SWC chair – a statement piece that brings mid-century charm to any home or office, now offered with a range of customization options and contract-grade durability. Part of a beautiful family of desks, cabinets, and tables, this chair blends iconic design with modern comfort.

Key features
- Design-forward, mid-century inspired silhouette with versatile shell colors and base finishes.
- Flexible upholstery options: SWC-100 (plastic back/front) or SWC-110 (full upholstery) in 10 fabrics or 6 leathers.
- Base finishes: stainless steel, matte black, gloss white, or chrome.
- Comfort and adjustability: pneumatic height adjustment and a sturdy 5-wheel base.
- Seating options: armless or 8-position PU armrests.
- Floor-friendly options: soft or hard-floor casters.
- Suitable for home or business settings and qualified for contract use.
- Made in Italy with meticulous craftsmanship.

Specifications at a glance
- Construction: 5-wheel plastic-coated aluminum base; pneumatic heig

## Issue 1: The text is too long
- Limit the number of words/sentences/characters.

In [22]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

Mid-century inspired ergonomic office chair from Italy, featuring a cast-aluminum shell, 5-wheel base, and pneumatic height adjust. Choose plastic or full upholstery (10 fabrics, 6 leathers), armless or 8-position arms, and base finishes in stainless, matte black, gloss white, or chrome. Suitable for home, office, and contract use.


## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [23]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

Mid-century inspired chair with a cast aluminum shell, 10 mm thick, coated in modified PA6/PA66. HD36 foam seat. Upholstery SWC-100 or SWC-110 in 10 fabrics or 6 leathers; 8-position PU armrests. 5-wheel aluminum base, pneumatic height adjustment, and base finishes: stainless steel, matte black, gloss white, chrome.


In [24]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

Mid-century inspired office chair with a cast aluminum shell coated in modified nylon PA6/PA66. 10 mm shell, HD36 foam seat, and pneumatic height adjustment. Exchangeable upholstery options and four base finishes (stainless steel, matte black, gloss white, chrome). Upholstery options SWC-100 or SWC-110


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [25]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

After the description, include a table that gives the
product's dimensions. The table should have two columns.
In the first column include the name of the dimension.
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website.
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div class="product-description">
  <p>Part of a beautiful family of mid-century inspired office furniture, this chair is designed to complement a broad range of settings—from private offices to contract environments. The collection includes filing cabinets, desks, bookcases, meeting tables, and more, with interchangeable shell colors and base finishes to tailor the look.</p>

  <p>Materials and construction are oriented toward durability and a premium tactile experience. The shell is a cast aluminum base with a modified nylon PA6/PA66 coating, delivering a robust, lightly crested silhouette. The shell thickness is 10 mm, contributing to structural rigidity without excess weight. The seat uses HD36 foam for comfortable resilience over long durations.</p>

  <p>Upholstery and customization options provide versatility for both residential and commercial applications. The chair is available with plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 le

In [26]:
from IPython.display import display, HTML

In [27]:
display(HTML(response))

Dimension,Inches
Width,20.87
Depth,20.08
Height,31.50
Seat Height,17.32
Seat Depth,16.14


# III - Summarizing
Summarize text with a focus on specific topics.

In [28]:
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \
super cute, and its face has a friendly look. It's \
a bit small for what I paid though. I think there \
might be other options that are bigger for the \
same price. It arrived a day earlier than expected, \
so I got to play with it myself before I gave it \
to her.
"""

## Summarize with a word/sentence/character limit

In [29]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site.

Summarize the review below, delimited by triple
backticks, in at most 30 words. .

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

```
A soft, cute panda plush that my daughter loves; friendly face. Slightly small for the price, with bigger options available. Arrived early, and I got to try it before gifting.
```


## Summarize with a focus on shipping and delivery

In [30]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment.

Summarize the review below, delimited by triple
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

```
Delivery was earlier than expected, arriving a day ahead of schedule and letting the reviewer enjoy it briefly before gifting.
```


## Summarize with a focus on price and value

In [31]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the \
price of the product.

Summarize the review below, delimited by triple
backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

```
Soft and loved by daughter, but it's small for the price, and bigger options exist at the same price, lowering perceived value despite early delivery.
```


## Try "extract" instead of "summarize"  
Summaries sometimes include aspects which are not needed.  

In [32]:
prompt = f"""
Your task is to extract relevant information from \
a product review from an ecommerce site to give \
feedback to the Shipping department.

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \
delivery. Limit to 30 words.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

Arrived a day earlier than expected; delivery was earlier than estimated, allowing me to use it before gifting.


## Summarize multiple product reviews

In [33]:
review_1 = prod_review

# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products.
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean!
"""

# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

reviews = [review_1, review_2, review_3, review_4]

In [34]:
for i in range(len(reviews)):
    prompt = f"""
    Your task is to generate a short summary of a product \
    review from an ecommerce site.

    Summarize the review below, delimited by triple \
    backticks in at most 20 words.

    Review: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

0 ```
Soft, cute panda plush loved by daughter, who carries it everywhere; small for price; early delivery; bigger options exist.
``` 

1 ```
Affordable storage lamp; fast shipping, responsive replacements for damaged or missing parts; easy assembly and excellent customer service.
``` 

2 ```
Impressive battery life and dentist clean feel; head too small; OEM heads pricey, generics cheaper; good value around $50.
``` 

3 ```
Notes price gouging after sale, declining product quality, motor failure after a year, warranty expired; replacement needed; brand loyalty concerns.
``` 



# IV - Inferring
Now we will infer sentiment and topics from product reviews and news articles.

In [35]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [36]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""

response = get_completion(prompt)
print(response)

Positive sentiment. The review expresses satisfaction with the lamp, price, fast shipping, easy assembly, and responsive customer service.


In [37]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Give your answer as a single word, either "Positive" \
or "Negative".

Review text: '''{lamp_review}'''
"""

response = get_completion(prompt)
print(response)

Positive


## Identify types of emotions

In [38]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""

response = get_completion(prompt)
print(response)

satisfaction, relief, gratitude, appreciation, trust


## Identify anger

In [39]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""

response = get_completion(prompt)
print(response)

no


## Extract product and company name from customer reviews

In [40]:
prompt = f"""
Identify the following items from the review text:
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.

Review text: '''{lamp_review}'''
"""

response = get_completion(prompt)
print(response)

{"Item":"lamp with storage","Brand":"Lumina"}


## Doing multiple tasks at once

In [41]:
prompt = f"""
Identify the following items from the review text:
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""

response = get_completion(prompt)
print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}


## Inferring topics

In [42]:
story = """
In a recent survey conducted by the government,
public sector employees were asked to rate their level
of satisfaction with the department they work at.
The results revealed that NASA was the most popular
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings,
stating, "I'm not surprised that NASA came out on top.
It's a great place to work with amazing people and
incredible opportunities. I'm proud to be a part of
such an innovative organization."

The results were also welcomed by NASA's management team,
with Director Tom Johnson stating, "We are thrilled to
hear that our employees are satisfied with their work at NASA.
We have a talented and dedicated team who work tirelessly
to achieve our goals, and it's fantastic to see that their
hard work is paying off."

The survey also revealed that the
Social Security Administration had the lowest satisfaction
rating, with only 45% of employees indicating they were
satisfied with their job. The government has pledged to
address the concerns raised by employees in the survey and
work towards improving job satisfaction across all departments.
"""

##### We have chosen to infer 5 topics

In [43]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long.

Format your response as a list of items separated by commas

Text sample: '''{story}'''
"""

response = get_completion(prompt)
print(response)

job satisfaction, NASA, public sector, survey, departments


##### Guiding the topic extraction:

In [44]:
topic_list = ["nasa", "local government", "engineering",
              "employee satisfaction", "federal government"
              ]

prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""

response = get_completion(prompt)
print(response)

[1, 0, 0, 1, 1]


# V - Transforming

Now we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

## Translation

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.

In [45]:
prompt = f"""
Translate the following English text to Spanish: \
```Hi, I would like to order a blender```
"""

response = get_completion(prompt)
print(response)

Hola, me gustaría pedir una licuadora.


In [46]:
prompt = f"""
Tell me which language is this:
```Combien coûte le lampadaire?```
"""

response = get_completion(prompt)
print(response)

French.

Translation: "How much does the streetlamp cost?"

Note: It uses the circumflex in coûte (û).


In [47]:
prompt = f"""
Translate the following  text to French, Chinese and Spanish
and English pirate: \
```I want to order a basketball```
"""

response = get_completion(prompt)
print(response)

- French: Je veux commander un ballon de basket.
- Chinese: 我想订购一个篮球。
- Spanish: Quiero pedir una pelota de baloncesto.
- English pirate: Arrr, I be wantin' to order a basketball.


In [48]:
prompt = f"""
Translate the following text to Germah in both the \
formal and informal forms:
'Would you like to order a pillow?'
"""

response = get_completion(prompt)
print(response)

It seems you might mean German rather than a language called “Germah.” If you intended German, here are the translations in formal and informal forms:

- Formal: Möchten Sie ein Kissen bestellen?
- Informal: Möchtest du ein Kissen bestellen?

If you did mean a different language named Germah, please share a bit more about its grammar or provide a sample, and I’ll translate accordingly.


### Universal Translator
Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [49]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
]

In [50]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """

    response = get_completion(prompt)
    print(response, "\n")

Original message (French. (Translation: "The system performance is slower than usual.")): La performance du système est plus lente que d'habitude.
- English: The system's performance is slower than usual.
- Korean: 시스템의 성능이 평소보다 느립니다. 

Original message (Spanish (Español). Translation: "My monitor has pixels that don’t light up."): Mi monitor tiene píxeles que no se iluminan.
English: My monitor has pixels that do not light up.
Korean: 제 모니터에는 빛이 들어오지 않는 픽셀들이 있습니다. 

Original message (Italian. It translates to "My mouse doesn’t work."): Il mio mouse non funziona
- English: My mouse isn't working.
- Korean: 제 마우스가 작동하지 않습니다. 

Original message (Polish.

Translation: "My Ctrl key is broken."

Notes: "Mój" = my (masc. possessive), "klawisz" = key, "jest zepsuty" = is broken (masc.).): Mój klawisz Ctrl jest zepsuty
- English: My Ctrl key is broken.
- Korean: 내 Ctrl 키가 망가졌어요. 

Original message (It's Chinese, specifically Simplified Chinese (Mandarin). The sentence means: "My screen is flic

## Tone Transformation
Writing can vary based on the intended audience. ChatGPT can produce different tones.

In [51]:
prompt = f"""
Translate the following from slang to a business letter:
'Dude, This is Joe, check out this spec on this standing lamp.'
"""

response = get_completion(prompt)
print(response)

Subject: Standing Lamp Specification

Dear [Recipient Name],

My name is Joe [Last Name]. I am writing to share the specification for the standing lamp. Please review the attached specification at your convenience. If you have any questions or require further details, I would be happy to discuss.

Best regards,
Joe [Last Name]
[Title], [Company]
[Phone]
[Email]


## Format Conversion
ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [52]:
data_json = { "resturant employees" :[
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""

response = get_completion(prompt)
print(response)

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>resturant employees</title>
</head>
<body>
  <h2>resturant employees</h2>
  <table border="1" cellpadding="5" cellspacing="0" aria-label="resturant employees">
    <thead>
      <tr>
        <th>name</th>
        <th>email</th>
      </tr>
    </thead>
    <tbody>
      <tr><td>Shyam</td><td>shyamjaiswal@gmail.com</td></tr>
      <tr><td>Bob</td><td>bob32@gmail.com</td></tr>
      <tr><td>Jai</td><td>jai87@gmail.com</td></tr>
    </tbody>
  </table>
</body>
</html>


In [53]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

name,email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response.

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [54]:
text = [
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use
    any punctuation around the text:
    ```{t}```"""

    response = get_completion(prompt)
    print(response)

The girl with the black and white puppies has a ball.
No errors found
It's going to be a long day. Does the car need its oil changed?
There goes my freedom. They're going to bring their suitcases.
You're going to need your notebook.
That medicine affects my ability to sleep. Have you heard of the butterfly effect?
This phrase is to check ChatGPT for spelling ability


In [55]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"

response = get_completion(prompt)
print(response)

Here is a cleaned-up version:

I got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears sits a bit lower than the other, which I don't think was designed to be asymmetrical. It's a bit small for what I paid, though. I think there are bigger options for the same price. It arrived a day earlier than expected, so I got to play with it myself before giving it to my daughter.


In [56]:
prompt = f"""
proofread and correct this review. Make it more compelling.
Ensure it follows APA style guide and targets an advanced reader.
Output in markdown format.
Text: ```{text}```
"""

response = get_completion(prompt)
display(Markdown(response))

# Panda Plush Toy: Review

## Overview
I purchased this panda plush for my daughter's birthday because she frequently borrows mine from my room. Adults also appreciate panda-themed items. She now carries the plush everywhere; it is exceptionally soft and visually appealing.

## Design, Comfort, and Aesthetics
The toy is soft to the touch and charming in appearance. One ear sits slightly lower than the other, indicating a minor manufacturing irregularity rather than an intentional asymmetrical design. This small defect does not detract significantly from overall appeal, but it may be noticeable to discerning buyers.

## Size and Value
Relative to the price, the plush feels small. There may be larger options available at a similar price point, which could better suit buyers prioritizing presence or heft.

## Delivery and Practical Use
The package arrived one day earlier than expected, allowing me to test the item briefly before presenting it to my daughter. This prompt delivery enhanced the gifting experience.

## Overall Assessment and Recommendation
Overall, the panda plush delivers desirable softness and charm, making it a compelling gift for a child who values cuddliness. However, potential buyers should consider the modest size and the minor symmetry irregularity when evaluating value. Rating: 4 out of 5 stars. Suitable for readers seeking a cuddly, endearing plush with a quick delivery experience; consider comparing sizes to ensure the chosen option meets expectations.

# VI - Expanding
Now you will generate customer service emails that are tailored to each customer's review.

## Customize the automated reply to a customer email

In [57]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [58]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service.
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""

response = get_completion(prompt)
print(response)

Subject: We apologize for your recent experience and want to help

Hello,

Thank you for sharing your detailed review. I’m sorry to hear about the issues you described—price changes from November’s $49 for the 17-piece system to December’s $70–$89, the $10 increase for the 11-piece system, concerns about the base blade locking, the motor noise after about a year, and the warranty having expired before you needed support. I also note you received the product in about two days, which you mentioned in the review.

We want to make this right. Please reach out to our customer service team so we can review your case and discuss options, including any warranty or replacement considerations. If possible, include your order number and a brief summary of the issues you encountered.

Thank you again for your feedback. We value your experience and will use it to improve.

Sincerely,
AI customer agent


## Remind the model to use details from the customer's email

In [59]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service.
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""


response = get_completion(prompt)
print(response)

Subject: We’re sorry for your experience with our blender systems

Hello,

Thank you for taking the time to share your detailed review. We’re genuinely sorry for the frustration you’ve described, including the pricing changes, the blade-lock base quality, and the motor noise after about a year, as well as the expired warranty that led to a replacement.

We note your specifics: the 17-piece system on sale for around $49 in November, the December price increase to about $70–$89, and the 11-piece system rising by about $10 from $29. We’ll forward your feedback to the relevant teams. Your smoothie and freezing tips are also appreciated.

To help us assist you further, please reply with your order number and the model, and a brief description of the motor noise. Our customer service team can review options, including any warranty coverage or goodwill resolution where applicable.

Thank you again for your feedback and for choosing us.

Sincerely,
AI customer agent


# VI - Using messages

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

In [61]:
def get_completion(prompt, model=model):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
        ]
    response = client.chat.completions.create(model=model,
                                              messages=messages,
                                              )
    return response.choices[0].message.content


def get_completion2(messages, model=model):
    response = client.chat.completions.create(model=model,
                                              messages=messages,
                                              )
    return response.choices[0].message.content

In [62]:
messages =  [
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
{'role':'user', 'content':'tell me a joke'},
{'role':'assistant', 'content':'Why did the chicken cross the road'},
{'role':'user', 'content':'I don\'t know'}  ]

response = get_completion2(messages=messages)
print(response)

Wherefore did the chicken cross the road? To get thee to the other side, good sir—a jest as simple as a feather and as merry as a summer’s sunrise. Want another, perchance, in this fair tongue?


In [65]:
response = get_completion2(messages=messages, model="gpt-5-mini")
print(response)

To get to the other side — forsooth!


In [66]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'}  ]

response = get_completion2(messages=messages)
print(response)

Hi Isa! Nice to meet you. How can I help today? I can answer questions, help with writing, brainstorm ideas, plan a trip or project, explain concepts, practice a language, or just chat about topics you enjoy. What would you like to do?


In [67]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]

response = get_completion2(messages=messages)
print(response)

I don’t know your name yet. If you tell me what you’d like to be called, I’ll use that in this chat. For example, you can say, "My name is Maya" or "Call me Alex." I can also remember a nickname for this session if you want. Note: I don’t store personal data between sessions unless your app supports memory and you opt in.

What name should I use for you?


In [69]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]

response = get_completion2(messages=messages)
print(response)

Your name is Isa.

Would you like me to keep using that name for the rest of this chat? I can refer to you as Isa. If you want me to remember it for future sessions, I’d need memory access to be enabled on your platform.


In [70]:
from IPython.display import clear_output

In [71]:
# Initialize the chat messages history
messages = [{"role": "assistant", "content": "How can I help?"}]

# Function to display the chat history
def display_chat_history(messages):
    for message in messages:
        print(f"{message['role'].capitalize()}: {message['content']}")

# Main chat loop
while True:
    # Display chat history
    clear_output()
    display_chat_history(messages)

    # Get user input
    prompt = input("User: ")
    if prompt == "":
        print("See you soon!")
        break
    messages.append({"role": "user", "content": prompt})

    # Get assistant response
    response = get_completion2(messages=messages)
    messages.append({"role": "assistant", "content": response})

Assistant: How can I help?
User: how can you help me?
Assistant: I can help with a lot of things. Here are some of the main ways I can assist:

- Information and explanations
  - Explain concepts in simple or detailed terms
  - Compare options, pros/cons, and trade-offs
  - Help with math, science, history, etc.
  - Summarize articles or documents

- Writing and editing
  - Draft emails, letters, resumes, cover letters, essays
  - Edit for clarity, grammar, tone, and structure
  - Create outlines, summaries, or creative writing prompts
  - Translate or simplify text

- Coding and technology
  - Write, debug, and explain code (Python, JavaScript, SQL, etc.)
  - Explain algorithms and data structures
  - help with regex, APIs, and basic software architecture

- Data, analysis, and planning
  - Analyze datasets, perform basic statistics, and visualize ideas (text-based)
  - Create charts, dashboards, or simple Excel/Sheets formulas
  - Plan projects, itineraries, study schedules, or meeti